In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ev2gym.models.ev2gym_env import EV2Gym


In [23]:
# gather data for forecasting purpose

from ev2gym.baselines.heuristics import ChargeAsFastAsPossible

config_file = "optimization_for_forecast.yaml"
env_for_forecast = EV2Gym(config_file=config_file,
              save_replay=False,
              save_plots=False)


# use random algo for the agent. We just want to gather the data of solar power and load for the previous 5 weekdays (last week)
# look into optimization_for_forecast.yaml for more info
agent = ChargeAsFastAsPossible()
for t in range(env_for_forecast.simulation_length):
    # get action
    actions = agent.get_action(env_for_forecast)
    env_for_forecast.step(actions)

# this simulation assume 1 transformer which acts as the grid, so I can directly use an index of 0 below
solar_power_last_week = env_for_forecast.tr_solar_power[0, :] * -1 # negative sign to make it positive.  because this simulator shows solar power as negative
load_last_week = env_for_forecast.tr_inflexible_loads[0, :]

# save this data into a csv file
df = pd.DataFrame({'solar_power': solar_power_last_week, 'load': load_last_week})

df.to_csv('data_for_forecast.csv', index=False)